Orphaned notebook with values to "glue" using [myst-nb](https://myst-nb.readthedocs.io/en/v0.13.2/use/glue.html).

In [1]:
import os

import myst_nb

import penzai
from penzai import pz

import jax
import jax.numpy as jnp
import orbax.checkpoint
from jax.experimental import mesh_utils

import IPython.utils.capture

/usr/local/lib/python3.10/dist-packages/jax/_src/cloud_tpu_init.py:73: UserWarning: JAX_USE_PJRT_C_API_ON_TPU no longer has an effect (the new TPU runtime is always enabled now). Unset the environment variable to disable this warning.
  warnings.warn(


In [2]:
pz.ts.register_as_default(streaming=False)
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from penzai.example_models import gemma

In [4]:
try:
  import kagglehub
  if jax.devices()[0].platform == "tpu":
    load_gemma = True
  else:
    load_gemma = False
  load_gemma = True
except ImportError:
  kagglehub = None
  load_gemma = False

In [5]:
if load_gemma:
  weights_dir = kagglehub.model_download('google/gemma/Flax/2b')
  ckpt_path = os.path.join(weights_dir, '2b')

  checkpointer = orbax.checkpoint.PyTreeCheckpointer()
  metadata = checkpointer.metadata(ckpt_path)

  n_devices = jax.local_device_count()
  sharding_devices = mesh_utils.create_device_mesh((n_devices,))
  sharding = jax.sharding.PositionalSharding(sharding_devices)
  restore_args = jax.tree_util.tree_map(
      lambda m: orbax.checkpoint.ArrayRestoreArgs(
          restore_type=jax.Array,
          sharding=sharding.reshape((1,) * (len(m.shape) - 1) + (n_devices,))
      ),
      metadata,
  )
  flat_params = checkpointer.restore(ckpt_path, restore_args=restore_args)

  model = gemma.model_core.GemmaTransformer.from_pretrained(
      flat_params, upcast_activations_to_float32=False
  )

else:
  model = pz.nn.initialize_parameters(
      gemma.model_core.GemmaTransformer.from_config(
          gemma.model_core.GemmaTransformerConfig(
              num_heads=8,
              embedding_dim=256,
              projection_dim=32,
              single_kv_head=False,
              mlp_hidden_dim=512,
              num_decoder_blocks=10,
              vocab_size=1000,
              parameter_dtype=jnp.float32,
              activation_dtype=jnp.float32,
          )
      ),
      jax.random.key(1),
  )

  0%|          | 0.00/3.66G [00:00<?, ?B/s]

  0%|          | 9.00M/3.66G [00:00<00:47, 82.7MB/s]

  1%|          | 25.0M/3.66G [00:00<00:31, 124MB/s] 

  1%|          | 37.0M/3.66G [00:00<00:43, 89.3MB/s]

  1%|▏         | 49.0M/3.66G [00:00<00:38, 100MB/s] 

...

 98%|█████████▊| 3.59G/3.66G [00:22<00:00, 246MB/s]

 99%|█████████▊| 3.61G/3.66G [00:23<00:00, 253MB/s]

 99%|█████████▉| 3.64G/3.66G [00:23<00:00, 256MB/s]

100%|██████████| 3.66G/3.66G [00:23<00:00, 169MB/s]


Extracting model files...


In [6]:
%%autovisualize

with IPython.utils.capture.capture_output() as capturer:
  pz.select(model).at(lambda root: (
      root.body.body.body.sublayers[2].sublayers[0].delta.sublayers[1].input_to_query,
      root.body.body.body.sublayers[2].sublayers[1].delta.sublayers[1],
  )).show_value()

In [7]:
myst_nb.glue(
    "penzai_teaser",
    IPython.display.HTML(
        "".join(output.data['text/html'] for output in capturer.outputs)
    ),
)